#### websitesinde oldukça fazla örnek bulunduğundan -projede hepsi çekilse de- göstermelik sadece ilk 4 mekan çekilerek gösterilmiştir.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pickle

import json
import requests
import csv
import pandas as pd
import numpy as np

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import warnings
warnings.filterwarnings("ignore") 

## All URL's from main pages 

In [8]:
def get_URLs(page_num):
    
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")

    DRIVER_PATH = 'chromedriver.exe'
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

    places = {}

    # 34 sayfa gezilecek
    for i in range(page_num):
        
        # her sayfadan 100 link çekilecek 
        mainurl = 'https://www.kulturportali.gov.tr/turkiye/genel/gezilecekyer?sayfa=' + str(i+1) + '&sayi=4'
        driver.get(mainurl)

        try:
            WebDriverWait(driver, 10).until(lambda driver: len(
                driver.find_elements(By.XPATH, "//article[@class='portfolio-item']")) == 4)
        except TimeoutException:
            print("Loading took too much time!")

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # link ekleri çekilecek
        for div in soup.find_all('div', {'class': 'portfolio-desc'}):
            places[div.find('h3').text] = div.find('a')['href']

    driver.quit()
    return places

In [3]:
def print_places(places):
    for place, url in places.items():
        print(place, ": ", url, "\n")

In [4]:
def save_json(file_name, dic):
    with open(file_name, 'wb') as fp:
        pickle.dump(dic, fp)

In [5]:
def read_json(file_name):
    with open(file_name, 'rb') as fp:
        data = pickle.load(fp)
    return data

In [9]:
page_num = 1
places = get_URLs(page_num)

Loading took too much time!


In [10]:
print_places(places)

SÜMELA MANASTIRI :  /turkiye/trabzon/gezilecekyer/meryem-ana-sumela-manastiri 

IHLARA VADİSİ :  /turkiye/aksaray/gezilecekyer/ihlara-vadisi 

DARA ANTİK KENTİ :  /turkiye/mardin/gezilecekyer/dara-antik-kenti 

KIZ KALESİ  :  /turkiye/mersin/gezilecekyer/kizkalesi-deniz-kalesi 



In [ ]:
save_json('place_url.json', places)

In [ ]:
place_url = read_json('place_url.json')
names = list(place_url.keys())
urls = list(place_url.values())
print(names)

In [14]:
for url in places.keys():
    print(url)

SÜMELA MANASTIRI
IHLARA VADİSİ
DARA ANTİK KENTİ
KIZ KALESİ 


In [15]:
for url in places.values():
    print(url)

/turkiye/trabzon/gezilecekyer/meryem-ana-sumela-manastiri
/turkiye/aksaray/gezilecekyer/ihlara-vadisi
/turkiye/mardin/gezilecekyer/dara-antik-kenti
/turkiye/mersin/gezilecekyer/kizkalesi-deniz-kalesi


In [ ]:
for url in urls:
    print(url)

In [ ]:
len(urls)

## Names - Descriptions - Labels

In [16]:
def get_URLs(names, urls):
    
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")

    DRIVER_PATH = 'chromedriver.exe'
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

    places_desc = []
    i=0
    
    labels = []
    for url in urls:
        
        # önceki aşamada çekilen link ekleri main url'ye uçtan eklenerek mekan linkleri oluşturulacak        
        mainurl = 'https://www.kulturportali.gov.tr'
        mainurl += url
        driver.get(mainurl)
        print(mainurl)
        
        sentences = ""
        resp = requests.get(mainurl, verify=False).text
        soup = BeautifulSoup(resp)
        sub_texts = soup.find("div",{"id":"descriptionDiv"}).text
        
        for sub_text in sub_texts.split("\n"):
            
            # metnin bulunduğu kısımda istenmeyen ekler alınmayacak
            if not sub_text.startswith("Fotoğraf") and len(sub_text) > 2:
                if sub_text.startswith("Kaynak:"):
                    break
                sentences += sub_text            
                                            
        places_desc.append(sentences)
        i += 1
        
        # labels
        items = []
        get_info = iter(soup.find_all("div",{"class":"col-md-6 col-sm-12"}))
        for item in get_info:
            #label = item.find("label")
            #value = item.find("span")
                        
            if item.find("label").string == "Turizm Türü: ":
                labels.append(item.find("span").string)
                                            
    driver.quit()
    return places_desc, labels

In [ ]:
#deneme = urls[:5]

In [ ]:
places_desc, labels = get_URLs(names, urls)

In [17]:
places_desc, labels = get_URLs(places.keys(),places.values())

https://www.kulturportali.gov.tr/turkiye/trabzon/gezilecekyer/meryem-ana-sumela-manastiri
https://www.kulturportali.gov.tr/turkiye/aksaray/gezilecekyer/ihlara-vadisi
https://www.kulturportali.gov.tr/turkiye/mardin/gezilecekyer/dara-antik-kenti
https://www.kulturportali.gov.tr/turkiye/mersin/gezilecekyer/kizkalesi-deniz-kalesi


In [18]:
labels

['İnanç-Kültür Turizmi',
 'Doğa-Kültür Turizmi',
 'Kültür Turizmi',
 'Kültür Turizmi']

In [19]:
places_desc[0]

'Sümela Manastırı (Meryem Ana) Manastırı Nerededir?Trabzon’un Maçka İlçesi\'nin Altındere Köyü sınırları içinde, Altındere vadisine hakim Karadağ’ın eteklerinde sarp bir kayalık üzerine kurulmuş olan\xa0Sümela Manastırı, halk arasında "Meryem Ana" adı ile anılır. Vadiden yaklaşık 300 metre yükseklikte bulunan yapı, bu konumuyla manastırların şehir dışında, ormanlarda, mağara ve su kenarlarında kurulma geleneğini sürdürmüştür.\xa0Meryem Ana\xa0adına kurulan manastırın "Sümela" adını "siyah" anlamına gelen "melas" sözcüğünden aldığı söylenmektedir. Bu ismin manastırın kurulduğu koyu renkli Karadağlar’ dan geldiği düşünülmekte ise de Sümela kelimesi buradaki Meryem tasvirinin siyah rengine bağlanabilmektedir.Sümela Manastırı\'nın TarihiRivayete göre Bizans İmparatoru\xa0 I.Theodosius zamanında (375-395) Atina’dan gelen Barnabas ve Sophranios isimli iki rahip tarafından kurulmuş olan manastır, 6\'ıncı yüzyılda İmparator Justinianus’un manastırın onarılarak genişletilmesini istemesi üzerine

In [21]:
myDict = {"Place": places.keys(), "Description": places.values(), "Tourism": labels}
df = pd.DataFrame.from_dict(myDict)
df.head()

,Place,Description,Tourism
0,SÜMELA MANASTIRI,/turkiye/trabzon/gezilecekyer/meryem-ana-sumel...,İnanç-Kültür Turizmi
1,IHLARA VADİSİ,/turkiye/aksaray/gezilecekyer/ihlara-vadisi,Doğa-Kültür Turizmi
2,DARA ANTİK KENTİ,/turkiye/mardin/gezilecekyer/dara-antik-kenti,Kültür Turizmi
3,KIZ KALESİ,/turkiye/mersin/gezilecekyer/kizkalesi-deniz-k...,Kültür Turizmi


In [22]:
len(myDict['Place'])

4

In [ ]:
with open('my_df.json', 'w') as fp:
    json.dump(myDict, fp)

In [ ]:
with open('my_df.json', encoding='utf-8') as f:
    my_file = json.load(f)

In [ ]:
my_df = pd.DataFrame(my_file)
my_df

In [ ]:
np.unique(my_df['Tourism'])

In [ ]:
my_df[my_df['Tourism'] == 'Yayla Turizmi']

In [ ]:
myDict